# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from azureml.data.dataset_factory import TabularDatasetFactory
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


In [2]:
#Testing the authentication using the Workspace method "from_config"
Workspace.from_config()

Workspace.create(name='quick-starts-ws-137355', subscription_id='aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee', resource_group='aml-quickstarts-137355')

In [3]:
#create workspace
ws = Workspace.from_config()

#original code:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'your experiment name here'

experiment=Experiment(ws, experiment_name)

In [4]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'automl-experiment-1'
project_folder = './pipeline-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
automl-experiment-1,quick-starts-ws-137355,Link to Azure Machine Learning studio,Link to Documentation


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [5]:


dataset = TabularDatasetFactory.from_delimited_files('https://www.openml.org/data/get_csv/16826755/phpMYEkMl')


dataset = dataset.to_pandas_dataframe()
dataset.describe()

,pclass,survived,sibsp,parch,fare
count,1309.000000,1309.000000,1309.000000,1309.000000,1308.000000
mean,2.294882,0.381971,0.498854,0.385027,33.295479
std,0.837836,0.486055,1.041658,0.865560,51.758668
min,1.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,0.000000,0.000000,7.895800
50%,3.000000,0.000000,0.000000,0.000000,14.454200
75%,3.000000,1.000000,1.000000,0.000000,31.275000
max,3.000000,1.000000,8.000000,9.000000,512.329200


In [6]:
###Just in case:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           min_nodes=1,
                                                           max_nodes=6)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.

Running


### Clean Data:

In [7]:
from train import clean_data

In [8]:
x, y = clean_data(dataset)

In [9]:
from sklearn.preprocessing import StandardScaler

In [10]:
# create scaler
variables = x.columns.tolist()

scaler = StandardScaler()
scaler.fit(x[variables]) 

x = scaler.transform(x[variables])

In [11]:
x = pd.DataFrame(x,columns=variables)


In [12]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y,
                                                    test_size = 0.2,
                                                    random_state=0)

In [13]:
#bring them together them again
dataset = pd.concat([x_train,y_train],axis=1)

In [14]:
dataset.head()

,pclass,age,sibsp,parch,fare,age_NA,fare_NA,sex_male,cabin_Missing,cabin_Rare,embarked_Q,embarked_Rare,embarked_S,title_Mr,title_Mrs,title_Rare,survived
1118,0.841916,-0.349075,-0.479087,-0.445,-0.490240,-0.501432,-0.02765,0.743497,0.539377,-0.425920,-0.322040,-0.039118,0.657394,0.852592,-0.42592,-0.274947,0
44,-1.546098,0.891204,-0.479087,-0.445,1.956990,-0.501432,-0.02765,-1.344995,-1.853992,2.347858,-0.322040,-0.039118,-1.521159,-1.172894,-0.42592,-0.274947,1
1072,0.841916,-0.116523,-0.479087,-0.445,-0.493947,1.994288,-0.02765,0.743497,0.539377,-0.425920,3.105202,-0.039118,-1.521159,0.852592,-0.42592,-0.274947,0
1130,0.841916,-0.891698,-0.479087,-0.445,-0.493141,-0.501432,-0.02765,-1.344995,0.539377,-0.425920,-0.322040,-0.039118,0.657394,-1.172894,-0.42592,-0.274947,0
574,-0.352091,-0.039005,0.481288,-0.445,-0.237445,-0.501432,-0.02765,0.743497,0.539377,-0.425920,-0.322040,-0.039118,0.657394,0.852592,-0.42592,-0.274947,0


In [15]:
#To train the model we need a TabularDataset and not a dataframe, therefore the current df will be converterd 
#into a TabularDataset:

#Convert the dataframe into a csv
local_path = 'prepared.csv'

#Save it locally
dataset.to_csv(local_path,index=None)

#Generate the a datastore object which is the the default datastore
datastore = ws.get_default_datastore()

In [16]:
#Upload the dataframe which was previosly converted into a csv
datastore.upload(src_dir='.', target_path='data')

Uploading an estimated of 9 files
Uploading ./automl2.ipynb
Uploaded ./automl2.ipynb, 1 files out of an estimated total of 9
Uploading ./prepared.csv
Uploaded ./prepared.csv, 2 files out of an estimated total of 9
Uploading ./automl2.ipynb.amltmp
Uploaded ./automl2.ipynb.amltmp, 3 files out of an estimated total of 9
Uploading ./hyperparameter_tuning.ipynb
Uploaded ./hyperparameter_tuning.ipynb, 4 files out of an estimated total of 9
Uploading ./pre_preprocess.py
Uploaded ./pre_preprocess.py, 5 files out of an estimated total of 9
Uploading ./train.py
Uploaded ./train.py, 6 files out of an estimated total of 9
Uploading ./.ipynb_aml_checkpoints/automl2-checkpoint2021-1-5-17-52-6.ipynb
Uploaded ./.ipynb_aml_checkpoints/automl2-checkpoint2021-1-5-17-52-6.ipynb, 7 files out of an estimated total of 9
Uploading ./__pycache__/pre_preprocess.cpython-36.pyc
Uploaded ./__pycache__/pre_preprocess.cpython-36.pyc, 8 files out of an estimated total of 9
Uploading ./__pycache__/train.cpython-36.pyc

$AZUREML_DATAREFERENCE_62df14bf0c1c41bb8fda1ce6d8def34f

In [17]:
#For the sake of checking; check the path
datastore.path()

$AZUREML_DATAREFERENCE_workspaceblobstore

In [18]:
#Now the uploaded file will be transformed into a Tabular dataset and store in a varible named 'training_dataset'
training_dataset = Dataset.Tabular.from_delimited_files(path= [(datastore,('data/prepared.csv'))])

In [19]:
#let's visualize the data:
training_dataset.to_pandas_dataframe().head()

,pclass,age,sibsp,parch,fare,age_NA,fare_NA,sex_male,cabin_Missing,cabin_Rare,embarked_Q,embarked_Rare,embarked_S,title_Mr,title_Mrs,title_Rare,survived
0,0.841916,-0.349075,-0.479087,-0.445,-0.490240,-0.501432,-0.02765,0.743497,0.539377,-0.425920,-0.322040,-0.039118,0.657394,0.852592,-0.42592,-0.274947,0
1,-1.546098,0.891204,-0.479087,-0.445,1.956990,-0.501432,-0.02765,-1.344995,-1.853992,2.347858,-0.322040,-0.039118,-1.521159,-1.172894,-0.42592,-0.274947,1
2,0.841916,-0.116523,-0.479087,-0.445,-0.493947,1.994288,-0.02765,0.743497,0.539377,-0.425920,3.105202,-0.039118,-1.521159,0.852592,-0.42592,-0.274947,0
3,0.841916,-0.891698,-0.479087,-0.445,-0.493141,-0.501432,-0.02765,-1.344995,0.539377,-0.425920,-0.322040,-0.039118,0.657394,-1.172894,-0.42592,-0.274947,0
4,-0.352091,-0.039005,0.481288,-0.445,-0.237445,-0.501432,-0.02765,0.743497,0.539377,-0.425920,-0.322040,-0.039118,0.657394,0.852592,-0.42592,-0.274947,0


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [20]:
#Create the automl settings which will be used as argurments in the automl config
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}

#Create the automl_config
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=training_dataset,
                             label_column_name="survived",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [21]:
#New one
#Original code
# TODO: Submit your experiment
#remote_run = experiment.submit(automl_config)
automl_run = experiment.submit(automl_config,show_output=True)

Running on remote.
No run_configuration provided, running on auto-ml with default configuration
Running on remote compute: auto-ml
Parent Run ID: AutoML_0b31e7a1-a5f4-4cc3-bb7b-fcd2dceff51b

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|3                                |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, a

In [22]:
from azureml.widgets import RunDetails

In [23]:
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [24]:
# Retrieve and save your best automl model.

automl_run.get_output()

Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


(Run(Experiment: automl-experiment-1,
 Id: AutoML_0b31e7a1-a5f4-4cc3-bb7b-fcd2dceff51b_38,
 Type: azureml.scriptrun,
 Status: Completed),
 Pipeline(memory=None,
          steps=[('datatransformer',
                  DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                  feature_sweeping_config=None,
                                  feature_sweeping_timeout=None,
                                  featurization_config=None, force_text_dnn=None,
                                  is_cross_validation=None,
                                  is_onnx_compatible=None, logger=None,
                                  observer=None, task=None, working_dir=None)),
                 ('prefittedsoftvotingclassifier',...
                                                                                                         tol=0.0001,
                                                                                                         validation_fraction=0.

In [25]:
best_automl_run, best_model = automl_run.get_output()


Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


In [26]:
best_model

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                        tol=0.0001,
                                                                                                        validation_fraction=0.1,
                                                                                                        verbose=0,
                                

In [40]:
model = best_automl_run.register_model(model_name = "best_run_automl.pkl", model_path = './outputs/')

In [41]:
model

Model(workspace=Workspace.create(name='quick-starts-ws-137355', subscription_id='aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee', resource_group='aml-quickstarts-137355'), name=best_run_automl.pkl, id=best_run_automl.pkl:2, version=2, tags={}, properties={})

In [28]:
#Take a look to the parameters of the best run; eventually not needed
best_automl_run.get_tags()

{'_aml_system_azureml.automlComponent': 'AutoML',
 '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":1,"CurrentNodeCount":1}',
 'ensembled_iterations': '[19, 34, 1, 33, 20, 6, 16, 13, 7, 2, 10]',
 'ensembled_algorithms': "['LightGBM', 'XGBoostClassifier', 'XGBoostClassifier', 'LightGBM', 'LightGBM', 'XGBoostClassifier', 'LightGBM', 'ExtremeRandomTrees', 'SVM', 'RandomForest', 'GradientBoosting']",
 'ensemble_weights': '[0.07142857142857142, 0.07142857142857142, 0.21428571428571427, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.14285714285714285, 0.07142857142857142, 0.07142857142857142]',
 'best_individual_pipeline_score': '0.8147086914995224',
 'best_individual_iteration': '19',
 '_aml_system_automl_is_child_run_end_telemetry_event_logged': 'True'}

In [29]:
#eventually not needed
#Also take a look to the recommended function
from pprint import pprint

def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()

print_model(best_model)

datatransformer
{'enable_dnn': None,
 'enable_feature_sweeping': None,
 'feature_sweeping_config': None,
 'feature_sweeping_timeout': None,
 'featurization_config': None,
 'force_text_dnn': None,
 'is_cross_validation': None,
 'is_onnx_compatible': None,
 'logger': None,
 'observer': None,
 'task': None,
 'working_dir': None}

prefittedsoftvotingclassifier
{'estimators': ['19', '34', '1', '33', '20', '6', '16', '13', '7', '2', '10'],
 'weights': [0.07142857142857142,
             0.07142857142857142,
             0.21428571428571427,
             0.07142857142857142,
             0.07142857142857142,
             0.07142857142857142,
             0.07142857142857142,
             0.07142857142857142,
             0.14285714285714285,
             0.07142857142857142,
             0.07142857142857142]}

19 - minmaxscaler
{'copy': True, 'feature_range': (0, 1)}

19 - lightgbmclassifier
{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 0.6933333333333332,
 'importance_

In [46]:
print(best_automl_run.id)


AutoML_0b31e7a1-a5f4-4cc3-bb7b-fcd2dceff51b_38


In [48]:
#let's deploy since the automl is stronger thatn the hyperparametr model
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig


In [49]:
env = best_automl_run.get_environment()

In [50]:
for pip_package in ["scikit-learn"]:
    env.python.conda_dependencies.add_pip_package(pip_package)

inference_config = InferenceConfig(entry_script='score.py',environment=env)

ERROR:azureml._model_management._util:entry_script score.py doesn't exist. entry_script should be path relative to current working directory



WebserviceException: WebserviceException:
	Message: entry_script score.py doesn't exist. entry_script should be path relative to current working directory
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "entry_script score.py doesn't exist. entry_script should be path relative to current working directory"
    }
}

In [39]:
service = model.deploy(Workspace=ws,name = 'titanic best model',models = [model],inference_config = inference_config, deployment_config=acid_config,overwrite=True)

Help on Run in module azureml.core.run object:

class Run(azureml._run_impl.run_base._RunBase)
 |  Defines the base class for all Azure Machine Learning experiment runs.
 |  
 |  A *run* represents a single trial of an experiment. Runs are used to monitor the asynchronous
 |  execution of a trial, log metrics and store output of the trial, and to analyze results and access
 |  artifacts generated by the trial.
 |  
 |  Run objects are created when you submit a script to train a model in many different scenarios in
 |  Azure Machine Learning, including HyperDrive runs, Pipeline runs, and AutoML runs.
 |  A Run object is also created when you :func:`azureml.core.experiment.Experiment.submit` or
 |  :func:`azureml.core.experiment.Experiment.start_logging` with the :class:`azureml.core.Experiment` class.
 |  
 |  To get started with experiments and runs, see
 |  
 |  * `Create and manage environments for training and
 |    deployment <https://docs.microsoft.com/azure/machine-learning/how-t

### The above code is not needed anymore; only the very next line will be then matched befores we submit the experiment

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

In [ ]:
pipeline_run.wait_for_completion()

## Examine Results

### Retrieve the metrics of all child runs
Outputs of above run can be used as inputs of other steps in pipeline. In this tutorial, we will examine the outputs by retrieve output data and running some tests.

In [ ]:
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
metrics_output

In [ ]:
num_file_downloaded = metrics_output.download('.', show_progress=True)

In [ ]:
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
# Retrieve best model from Pipeline Run
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

In [ ]:
#TODO: Save the best model
import pickle

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

In [ ]:
best_model.steps

### Test the Model:

In [ ]:
from sklearn.metrics import confusion_matrix
ypred = best_model.predict(x_test)
cm = confusion_matrix(y_test, ypred)

In [ ]:
pd.DataFrame(cm).style.background_gradient(cmap='Blues', low=0, high=0.9)

In [ ]:
#Accuracy Test:
(152 + 62) / (152 + 62 + 38+10)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
best_model

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
#help(best_model)
dir(pipeline_run)

In [ ]:
pipeline_run.experiment

TODO: In the cell below, print the logs of the web service and delete the service